# Library

In [1732]:
import pandas as pd
import numpy as np
import os
import re
import collections
#from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Functions

In [1733]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [1734]:
# def filter_byindex(x,y):
#     #selectors = [x for x in col2]
#     return list(itertools.compress(x,y))

In [1735]:
def check_len(x,y):
    if x==y:
        return 0
    else:
        return 1

In [1736]:
def get_lf(i,j,k):
    if pd.isnull(k):
        regex_lf = re.compile(r'((?:\w+\W+){1,'+str(len(i))+'})\(\s'+i[0]+'.*\)')
        return regex_lf.findall(j)
    else:
        return k

In [1737]:
def ngram_filter(doc, word, n):
    tokens = doc.split()
    all_ngrams = ngrams(tokens, n)
    filtered_ngrams = [x for x in all_ngrams if word in x]
    return filtered_ngrams

In [1738]:
# def get_longform(tokens, acro, margin = 2, i =1):
#     long_form = ''
#     #Looking for before
#     for word in tokens[index-margin-len(acro):index]:
#         #if first letter of word is equal to first letter os acronym
#         if word[0] == acro[i].lower():
#             long_form += word + ' '
#             i += 1
#             if i == len(acro):
#                 break
#         elif (i == 1) and (word[0] == acro[i-1].lower()):
#             long_form = word + ' '
#             i = 1
#             if i == len(acro):
#                 break
#     long_form = long_form.rstrip()
#     return long_form
    

In [1739]:
def get_longform(tokens, acro):
    acro = acro.lower()
    long_form = ''
    margin = 2
    i =0
    #Looking for before
    if acro not in tokens:
        return -1
    else:
        index = tokens.index(acro)
        for word in tokens[index-margin-len(acro):index]:
            #if first letter of word is equal to first letter os acronym
            if word[0] == acro[i].lower():
                long_form += word + ' '
                i += 1
                if i == len(acro):
                    break
            elif (i == 1) and (word[0] == acro[i-1].lower()):
                long_form = word + ' '
                i = 1
                if i == len(acro):
                    break
        long_form = long_form.rstrip()
        return long_form


# Load Data

### Trainning

318 clinical cases

In [1740]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/txt/")

In [1741]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [1742]:
train_raw.head()

,doc_id,texto
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,..."
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...


In [1743]:
train_raw.shape

(318, 2)

# Sub-track 1

## Found abbreviations (Short Forms)

### Regex

In [1744]:
patron3 = r'[A-Z]{2,8}' #Letras mayúsculas entre 2 y 8. Probar (\s|\()[A-Z]{2,8}
patron4 = r'\s[a-z]{1,2}\s' #Entre 2 y 3 letras minusculas entre espacios
patron5 = r'\b[aA-zZ]{1,4}\-[aA-zZ]{1,4}\b' #mayúsuclas o minúsculas entre guiones
patron6 = r'[1-9]\s*[aA-zZ]{1,4}\/\b[aA-zZ]{1,4}\b' #Palabras divididas por / solo cuando las palabras no exceden de 4 caracteres
patron7 = r'[aA-zZ]{1,4}[A-Z]+[a-z]*[1-4]*'
#patron8 = r'\/[a-z]*[A-Z]*'


# create a list with them
regexes = [ patron3, patron4, patron5, patron6, patron7]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s" % (patron3, patron4, patron5, patron6, patron7))

In [1745]:
#nltk.download('stopwords')
swords = list(set(stopwords.words('spanish')))

Add words to stopwords lists

In [1793]:
swords = swords + ['I','II','III','IV','V','VI','VII','VIII','IX', 'X', 'x']

### Get Short Formns with a regex in each text

In [1794]:
train_raw['abrev'] = train_raw['texto'].map(lambda x: generic_re.findall(x))

Separate mesurement units separate by "/"

In [1747]:
# train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [a.split("/") for a in x])

In [1748]:
# train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [item for sublist in x for item in sublist])

**Get offsets of the Short Forms founded in the text**

In [1795]:
train_raw['offse'] = train_raw['texto'].map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(generic_re, x)])

In [1796]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i.strip() for i in x])

Filter Short Forms to delete ones which are stopwords, get their index too for filter offsets lists later

In [1797]:
train_raw['abrev_index'] = train_raw['abrev'].apply(lambda x: [x.index(i) for i in x if i not in swords])

In [1798]:
train_raw['abrev'] = train_raw['abrev'].apply(lambda x: [i for i in x if i not in swords])

Filter offsets lists by index

In [1799]:
train_raw['offse'] = train_raw.apply(lambda x: [x['offse'][i] for i in x['abrev_index']], axis = 1)

In [1800]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len,len_check,texto_clean,tokens,zip
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[mm, DAKO, HHF]","[(185, 189), (1502, 1506), (1513, 1516)]","[8, 58, 59]",3,3,0,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","[(mm, (185, 189)), (DAKO, (1502, 1506)), (HHF,..."
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,...","[CT, MESNA]","[(408, 410), (1120, 1125)]","[14, 37]",3,3,0,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","[(CT, (408, 410)), (x, (445, 448)), (MESNA, (1..."
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...,[UIV],"[(400, 403)]",[8],1,1,0,mujer 29 años antecedentes ulcus duodenal estr...,"[mujer, 29, años, antecedentes, ulcus, duodena...","[(UIV, (400, 403))]"
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...,"[IRC, IgA, 8mg/Kg, 1mg/Kg, EEII, EEII, ROT, RC...","[(145, 148), (196, 199), (978, 984), (1030, 10...","[7, 9, 29, 31, 44, 44, 52, 54, 56, 61, 64, 68,...",17,17,0,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...","[(IRC, (145, 148)), (IgA, (196, 199)), (II, (5..."
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...,"[HDA, EEII, EID, EID, ROT, RCP, RMN, PEES, LCR...","[(234, 237), (829, 833), (866, 869), (866, 869...","[8, 33, 35, 35, 40, 42, 44, 51, 54, 55, 57, 55...",14,14,0,mujer 42 años momento someterse trasplante hep...,"[mujer, 42, años, momento, someterse, trasplan...","[(HDA, (234, 237)), (II, (276, 278)), (EEII, (..."


In [1801]:
train_raw[train_raw['doc_id'] == 'S0004-06142005001000011-1']['abrev']

3    [IRC, IgA, 8mg/Kg, 1mg/Kg, EEII, EEII, ROT, RC...
Name: abrev, dtype: object

Check abrev and offse columns has the same length

In [1811]:
train_raw['abrev_len'] =train_raw['abrev'].str.len()
train_raw['offse_len'] =train_raw['offse'].str.len()

In [1812]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len,len_check,texto_clean,tokens,zip
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[mm, DAKO, HHF]","[(185, 189), (1502, 1506), (1513, 1516)]","[8, 58, 59]",3,3,0,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","[(mm, (185, 189)), (DAKO, (1502, 1506)), (HHF,..."
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,...","[CT, MESNA]","[(408, 410), (1120, 1125)]","[14, 37]",2,2,0,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","[(CT, (408, 410)), (x, (445, 448)), (MESNA, (1..."
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...,[UIV],"[(400, 403)]",[8],1,1,0,mujer 29 años antecedentes ulcus duodenal estr...,"[mujer, 29, años, antecedentes, ulcus, duodena...","[(UIV, (400, 403))]"
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...,"[IRC, IgA, 8mg/Kg, 1mg/Kg, EEII, EEII, ROT, RC...","[(145, 148), (196, 199), (978, 984), (1030, 10...","[7, 9, 29, 31, 44, 44, 52, 54, 56, 61, 64, 68,...",16,16,0,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...","[(IRC, (145, 148)), (IgA, (196, 199)), (II, (5..."
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...,"[HDA, EEII, EID, EID, ROT, RCP, RMN, PEES, LCR...","[(234, 237), (829, 833), (866, 869), (866, 869...","[8, 33, 35, 35, 40, 42, 44, 51, 54, 55, 57, 55...",13,13,0,mujer 42 años momento someterse trasplante hep...,"[mujer, 42, años, momento, someterse, trasplan...","[(HDA, (234, 237)), (II, (276, 278)), (EEII, (..."


Check both lists have the same lenght

In [1813]:
train_raw['len_check'] = train_raw.apply(lambda row: check_len(row['abrev_len'],row['offse_len']),axis = 1)

Delete rows with different lenghts (check it later)

In [1814]:
train_raw.shape

(318, 11)

In [1815]:
train_raw[train_raw['len_check'] != 1].shape

(318, 11)

In [1761]:
#train_raw = train_raw[train_raw['len_check'] != 1]

### Text treatment

Remove string punctuation, lowecase, tokenize and remove stopwords

In [1816]:
for i in punctuation:
    train_raw['texto_clean'] = train_raw['texto'].str.replace(i,'').str.lower()

In [1817]:
train_raw['texto_clean'] = train_raw['texto_clean'].str.split().map(lambda x: ' '.join([w for w in x if w not in swords]))

In [1818]:
train_raw['tokens'] = train_raw['texto_clean'].map(lambda x: word_tokenize(x))

In [1819]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len,len_check,texto_clean,tokens,zip
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[mm, DAKO, HHF]","[(185, 189), (1502, 1506), (1513, 1516)]","[8, 58, 59]",3,3,0,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","[(mm, (185, 189)), (DAKO, (1502, 1506)), (HHF,..."
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,...","[CT, MESNA]","[(408, 410), (1120, 1125)]","[14, 37]",2,2,0,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","[(CT, (408, 410)), (x, (445, 448)), (MESNA, (1..."
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...,[UIV],"[(400, 403)]",[8],1,1,0,mujer 29 años antecedentes ulcus duodenal estr...,"[mujer, 29, años, antecedentes, ulcus, duodena...","[(UIV, (400, 403))]"
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...,"[IRC, IgA, 8mg/Kg, 1mg/Kg, EEII, EEII, ROT, RC...","[(145, 148), (196, 199), (978, 984), (1030, 10...","[7, 9, 29, 31, 44, 44, 52, 54, 56, 61, 64, 68,...",16,16,0,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...","[(IRC, (145, 148)), (IgA, (196, 199)), (II, (5..."
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...,"[HDA, EEII, EID, EID, ROT, RCP, RMN, PEES, LCR...","[(234, 237), (829, 833), (866, 869), (866, 869...","[8, 33, 35, 35, 40, 42, 44, 51, 54, 55, 57, 55...",13,13,0,mujer 42 años momento someterse trasplante hep...,"[mujer, 42, años, momento, someterse, trasplan...","[(HDA, (234, 237)), (II, (276, 278)), (EEII, (..."


### Get abreviations and offsets

Remove punctuation from abreviations

In [1766]:
# for i in punctuation:
#     train_raw['abrev'] = train_raw[train_raw['abrev'].notnull()]['abrev'].apply(lambda x: [a.replace(i,"") for a in x])

In [1820]:
train_raw['zip'] = train_raw.apply(lambda row: list(zip(row['abrev'], row['offse'])), axis = 1)

In [1821]:
train_raw.head()

,doc_id,texto,abrev,offse,abrev_index,abrev_len,offse_len,len_check,texto_clean,tokens,zip
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...,"[mm, DAKO, HHF]","[(185, 189), (1502, 1506), (1513, 1516)]","[8, 58, 59]",3,3,0,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","[(mm, (185, 189)), (DAKO, (1502, 1506)), (HHF,..."
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,...","[CT, MESNA]","[(408, 410), (1120, 1125)]","[14, 37]",2,2,0,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","[(CT, (408, 410)), (MESNA, (1120, 1125))]"
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...,[UIV],"[(400, 403)]",[8],1,1,0,mujer 29 años antecedentes ulcus duodenal estr...,"[mujer, 29, años, antecedentes, ulcus, duodena...","[(UIV, (400, 403))]"
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...,"[IRC, IgA, 8mg/Kg, 1mg/Kg, EEII, EEII, ROT, RC...","[(145, 148), (196, 199), (978, 984), (1030, 10...","[7, 9, 29, 31, 44, 44, 52, 54, 56, 61, 64, 68,...",16,16,0,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...","[(IRC, (145, 148)), (IgA, (196, 199)), (8mg/Kg..."
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...,"[HDA, EEII, EID, EID, ROT, RCP, RMN, PEES, LCR...","[(234, 237), (829, 833), (866, 869), (866, 869...","[8, 33, 35, 35, 40, 42, 44, 51, 54, 55, 57, 55...",13,13,0,mujer 42 años momento someterse trasplante hep...,"[mujer, 42, años, momento, someterse, trasplan...","[(HDA, (234, 237)), (EEII, (829, 833)), (EID, ..."


### Get one row per abbreviation

Separate elements lists in different rows

In [1822]:
mine = train_raw.explode('zip')

In [1823]:
mine.shape

(2880, 11)

In [1824]:
mine[mine['zip'].isnull()].shape

(19, 11)

In [1825]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'zip']]

In [1826]:
mine = mine[mine['zip'].notnull()]

In [1827]:
mine.shape

(2861, 4)

In [1828]:
mine.head()

,doc_id,texto_clean,tokens,zip
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","(mm, (185, 189))"
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","(DAKO, (1502, 1506))"
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","(HHF, (1513, 1516))"
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","(CT, (408, 410))"
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","(MESNA, (1120, 1125))"


### Separate SF from Offsets in different columns

In [1829]:
mine['abrev'] = mine.apply(lambda row: row['zip'][0], axis = 1)

In [1830]:
mine['offsets'] = mine.apply(lambda row: row['zip'][1], axis = 1)

In [1831]:
mine = mine[mine['abrev'] != ""]

In [1832]:
mine.head()

,doc_id,texto_clean,tokens,zip,abrev,offsets
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","(mm, (185, 189))",mm,"(185, 189)"
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","(DAKO, (1502, 1506))",DAKO,"(1502, 1506)"
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...","(HHF, (1513, 1516))",HHF,"(1513, 1516)"
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","(CT, (408, 410))",CT,"(408, 410)"
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...","(MESNA, (1120, 1125))",MESNA,"(1120, 1125)"


Separate offsets tuples in different columns

In [1833]:
mine[['startOffset', 'endOffset']] = pd.DataFrame(mine['offsets'].tolist(), index=mine.index) 

In [1834]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'abrev', 'startOffset', 'endOffset']]

In [1835]:
mine.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",mm,185,189
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",DAKO,1502,1506
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",HHF,1513,1516
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",CT,408,410
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",MESNA,1120,1125


Delete null values, and change ttype to integer for offsets columns

In [1836]:
mine.shape

(2861, 6)

In [1837]:
mine.dropna(subset=['startOffset', 'endOffset'], inplace = True)

In [1838]:
mine.shape

(2861, 6)

In [1839]:
mine['startOffset'] = mine['startOffset'].astype(int)
mine['endOffset'] = mine['endOffset'].astype(int)

In [1840]:
mine = mine[['doc_id', 'texto_clean', 'tokens', 'abrev', 'startOffset', 'endOffset']]

In [1841]:
mine.shape

(2861, 6)

In [1789]:
#mine[mine.duplicated(subset=['doc_id', 'texto_clean', 'abrev', 'startOffset', 'endOffset'], keep=False)]

In [1842]:
mine = mine.drop_duplicates(subset = ['doc_id', 'texto_clean', 'abrev', 'startOffset', 'endOffset'])

In [1843]:
mine.shape

(1996, 6)

In [1846]:
mine.head(50)

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",mm,185,189
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",DAKO,1502,1506
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",HHF,1513,1516
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",CT,408,410
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",MESNA,1120,1125
2,S0004-06142005000900016-1,mujer 29 años antecedentes ulcus duodenal estr...,"[mujer, 29, años, antecedentes, ulcus, duodena...",UIV,400,403
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",IRC,145,148
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",IgA,196,199
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",8mg/Kg,978,984
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",1mg/Kg,1030,1036


In [1730]:
# mine['abrev'] = mine['abrev'].apply(lambda x: [a.split("/") for a in x])

Check dataframes for one text

In [1406]:
# mine[mine['doc_id'] == 'S1130-05582012000300005-1']

In [1407]:
# train_abbr[train_abbr['doc_id'] == 'S1130-05582012000300005-1']

## Search Long Formns

### Search Long Forms in the same text

**measurement units dictionary**

In [1408]:
# mu_dic = {"mL":"Mililitro",
# "mg":"Miligramo",
# "g":"Gramo",
# "L":"Litro",
# "mcg":"Microgramo",
# "mmol":"Milimol",
# "UI":"Unidades Internacionales",
# "Miles UI":"Miles de Unidades Internacionales",
# "Millones UI":"Millones de Unidades Internacionales",
# "UFC":"Unidades Formadoras de Colonias",
# "mEq":"Miliequivalente",
# "ng":"Nanogramo",
# "Lf":"Unidad Floculante",
# "UFP":"Unidad Formadora de Placa",
# "DIC":"Dosis Infectante Mediana de Cultivo Celular 50% ",
# "DIT":"Dosis Infectante Mediana de Cultivo Tisular 50% ",
# "DI":"Dosis Infectante 50% ",
# "mol":"Peso Molecular Gramo ",
# "Eq":"Peso Equivalente Gramo ",
# "Dosis":"Dosis",
# "Almh":"Almohadilla",
# "Amp":"Ampolla",
# "Anl":"Anillo",
# "Bar":"Barra",
# "Bolsa":"Bolsa",
# "Cap":"Capsula",
# "Car":"Caramelo",
# "Carp":"Carpula",
# "Cart":"Cartucho",
# "Com":"Comprimido",
# "Dia":"Dia",
# "Fras":"Frasco",
# "Fras-Amp":"Frasco Ampolla ",
# "Grag":"Gragea",
# "Hora":"Hora",
# "Imp":"Implante",
# "Jab":"Jab¢n",
# "Jer":"Jeringa Prellenada ",
# "uL":"Microlitro",
# "Ovu":"Ovulo",
# "Parche":"Parche",
# "Past":"Pastilla",
# "Perl":"Perla",
# "Pil":"Pildora",
# "Pip":"Pipeta",
# "%":"Porcentaje",
# "Sach":"Sachet",
# "Sob":"Sobre",
# "Sup":"Supositorio",
# "Tab":"Tableta",
# "Troc":"Trocisco",
# "Vial":"Vial",
# "Kg":"Kilogramo",
# "Gal":"Galon",
# "Sis":"Sistema Terapeutico",
# "mCi":"miliCuries",
# "mBq":"milibequerel",
# "UEL":"Unidades ELISA ",
# "DL":"Dosis Letal",
# "U USP":"Unidades USP ",
# "U":"Unidades",
# "Rot":"Rotacaps",
# "CCID":"Dosis Infecciosa en Cultivo de Célula ",
# "U":"UNIDAD",
# "Otros":"Otros",
# "µ Ci":"mcroCuries",
# "Esp":"Esporas",
# "mcHA":"microgramos de HA ",
# "Gom":"Goma",
# "KIU":"Unidad Inhibidora de Calicreina ",
# "mcel":"Millones de Células ",
# "DU":"Unidades de Antigeno D",
# "Dil D2":"Dil D2",
# "Tin.Mad.":"Tintura Madre",
# "Dil D4":"Dil D4",
# "Dil D5":"Dil D5",
# "Dil D1":"Dil D1",
# "Dil D8":"Dil D8",
# "Dil D3":"Dil D3",
# "OU":"Unidad de Opacidad ",
# "mm": "milimetro",
# "dm": "decimetro",
# "cm": "centimetro"}

In [1409]:
# mine['long_form'] = mine['abrev'].map(mu_dic)

In [1410]:
# mine_lf = mine[mine['abrev'].str.len() > 1].reset_index()

### Get LF before SF

In [1411]:
# mine[mine['doc_id'] == 'S0004-06142005000900013-1'].iloc[0]['tokens']

With get_longform function, words befores SF that starts with SF letters will be detected

In [1847]:
mine['long_form'] = mine.apply(lambda row: get_longform(row['tokens'], row['abrev']), axis = 1)

In [1848]:
mine.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",mm,185,189,mismo
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",DAKO,1502,1506,demostró actina
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",HHF,1513,1516,-1
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",CT,408,410,
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",MESNA,1120,1125,


In [1849]:
mine.shape

(1996, 7)

### Study LF founded or not founded

Study SF without LF in the running text

In [1850]:
mine[mine['long_form'] == -1].shape

(610, 7)

Check how many texts don't have LF for the SF in the same text

In [1851]:
mine[(mine['abrev'].notnull()) & (mine['long_form']== '')]

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",CT,408,410,
1,S0004-06142005000900015-1,"varón 36 años, antecedentes interés, estudiado...","[varón, 36, años, ,, antecedentes, interés, ,,...",MESNA,1120,1125,
2,S0004-06142005000900016-1,mujer 29 años antecedentes ulcus duodenal estr...,"[mujer, 29, años, antecedentes, ulcus, duodena...",UIV,400,403,
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",IRC,145,148,
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",8mg/Kg,978,984,
...,...,...,...,...,...,...,...
317,S2340-98942015000100005-1,presentamos caso paciente 62 años diciembre 20...,"[presentamos, caso, paciente, 62, años, diciem...",IIIC,201,205,
317,S2340-98942015000100005-1,presentamos caso paciente 62 años diciembre 20...,"[presentamos, caso, paciente, 62, años, diciem...",CA,426,428,
317,S2340-98942015000100005-1,presentamos caso paciente 62 años diciembre 20...,"[presentamos, caso, paciente, 62, años, diciem...",TAC,465,468,
317,S2340-98942015000100005-1,presentamos caso paciente 62 años diciembre 20...,"[presentamos, caso, paciente, 62, años, diciem...",mmHg,1002,1006,


In [1852]:
lf_null = mine[(mine['abrev'].notnull()) & (mine['long_form'] == '')].shape[0]
lf_null

903

In [1853]:
print(f"LF has not be found in the same text where the SF is in {lf_null/mine.shape[0]*100: .2f}% of texts")

LF has not be found in the same text where the SF is in  45.24% of texts


**Create dictionary with SF and LF pairs founded in the same text**

In [1854]:
df_pairs = mine[(mine['abrev'].notnull()) & (mine['long_form'] != '')]

In [1855]:
df_pairs.head()

,doc_id,texto_clean,tokens,abrev,startOffset,endOffset,long_form
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",mm,185,189,mismo
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",DAKO,1502,1506,demostró actina
0,S0004-06142005000900013-1,trata mujer 29 años sometida estudio ecográfic...,"[trata, mujer, 29, años, sometida, estudio, ec...",HHF,1513,1516,-1
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",IgA,196,199,irc glomerulonefritis
3,S0004-06142005001000011-1,"varón 58 años edad momento trasplante, 5 octub...","[varón, 58, años, edad, momento, trasplante, ,...",EEII,1455,1459,extremidades


In [1856]:
pairs_dic = {}
for index, row in df_pairs.iterrows():
    if row['long_form'] != -1:
        if not row['abrev'] in pairs_dic:
            pairs_dic[row['abrev']] = set()
        pairs_dic[row['abrev']].add(row['long_form'])

In [1857]:
pairs_dic

{'mm': {'ma',
  'menos',
  'mismo',
  'mostraba',
  'mostró masa',
  'muestra',
  'máxima'},
 'DAKO': {'demostró actina'},
 'IgA': {'ig g', 'inmunoglobulinas', 'irc glomerulonefritis'},
 'EEII': {'exploración', 'extremidades'},
 'ROT': {'reflejo osteo-tendinoso', 'reflejos osteotendinosos'},
 'RCP': {'reflejo cutaneoplantar', 'rotulianos'},
 'RMN': {'realiza',
  'realización',
  'realizron',
  'realizó',
  'replecionada',
  'resonancia magnética nuclear',
  'romberg'},
 'LCR': {'libre', 'lumbar', 'líquido cefalorraquídeo'},
 'PCR': {'polimerasa', 'proteína c reactiva', 'protocolo'},
 'TSP': {'tropical'},
 'EID': {'eeii'},
 'PEES': {'parietal'},
 'RTU': {'realizo',
  'recibió tratamiento',
  'resección transuretral',
  'respondía tratamiento'},
 'FAV': {'fístula arteriovenosa'},
 'alfa-feto': {'análisis'},
 'Alfa-feto': {'análisis'},
 'PSA': {'posmiccional', 'presencia', 'procedió'},
 'CEA': {'carcino embrionario'},
 'FID': {'filiada'},
 'PAAF': {'posterior',
  'previamente',
  'profund

In [1858]:
sorted(pairs_dic.keys())

['A-P',
 'AA',
 'AAS',
 'AAT',
 'ACL',
 'AEO',
 'AFG',
 'AFP',
 'AGF',
 'AI',
 'AINES',
 'AL',
 'ALAT',
 'ALT',
 'AMA',
 'ANA',
 'ANCA',
 'ANCAS',
 'ANOES',
 'AO',
 'AP',
 'APC',
 'ASAT',
 'ASLO',
 'AST',
 'ATM',
 'AV',
 'AVI',
 'AVSC',
 'Alfa-feto',
 'BA',
 'BAL',
 'BAS',
 'BD',
 'BM',
 'BMC',
 'BMU',
 'BPA',
 'C-GSF',
 'C-Kit',
 'CA',
 'CD',
 'CDI',
 'CEA',
 'CK',
 'CMV',
 'COL',
 'CPK',
 'CT',
 'CU',
 'CX',
 'D-AAT',
 'DA',
 'DAI',
 'DAKO',
 'DFVA',
 'DG',
 'DII',
 'DM',
 'DP',
 'DPAR',
 'DR',
 'DRNS',
 'DTM',
 'EA',
 'EBHGA',
 'ECE',
 'ECG',
 'ECMO',
 'ECS',
 'EEII',
 'EF',
 'EFK',
 'EIA',
 'EID',
 'ELA',
 'EMA',
 'EMG',
 'ENA',
 'ENG',
 'EPOC',
 'EPR',
 'ESCHAP',
 'EVA',
 'FA',
 'FAV',
 'FID',
 'FMO',
 'FO',
 'FiO2',
 'GC',
 'GER',
 'GGT',
 'GOT',
 'GPT',
 'HBPM',
 'HCT',
 'HLA',
 'HPTS',
 'HPV',
 'HQR',
 'HSA',
 'HTA',
 'HTBI',
 'IAM',
 'ICG',
 'IF',
 'IGRA',
 'INSS',
 'IgA',
 'IgE',
 'IgG',
 'IgG4',
 'IgM',
 'LBA',
 'LCCT',
 'LCR',
 'LDH',
 'LH',
 'LLA',
 'LLBCP',
 'LLI',
 'LNH'

In [1423]:
len(pairs_dic.keys())

252

## Check text individually

In [1609]:
texto = train_raw.iloc[3]['texto']
texto

'Varón de 58 años de edad en el momento del trasplante, el 5 de octubre de 1998.\nEn programa de diálisis peritoneal desde enero del mismo año por IRC progresiva secundaria a glomerulonefritis tipo IgA con esclerosis focal y segmentaria asociada diagnosticada en 1984.\nOtros antecedentes de interés: hipertensión arterial diagnosticada en 1982, en tratamiento farmacológico, cardiopatía isquémica por enfermedad coronaria con angioplastia transluminal de la coronaria derecha en 1994 (desde entonces asintomático, pruebas de esfuerzo posteriores negativas), dislipemia tipo II. Intervenido de criptorquídia bilateral y amigdalectomizado en juventud.\nEl trasplante renal cursó sin incidencias (riñón izquierdo en fosa ilíaca derecha con 2 arterias en parche único y vena que se abocan a ilíacas externas), con función eficaz inmediata del injerto, sin rechazo agudo y niveles elevados de ciclosporina sin toxicidad clínica aparente. El paciente recibe tratamiento con ciclosporina 8mg/Kg, micofenola

In [1606]:
patron3 = r'[A-Z]{2,8}' #Letras mayúsculas entre 2 y 8. Probar (\s|\()[A-Z]{2,8}
patron4 = r'\s[a-z]{1,2}\s' #Entre 2 y 3 letras minusculas entre espacios
patron5 = r'\b[aA-zZ]{1,4}\-[aA-zZ]{1,4}\b' #mayúsuclas o minúsculas entre guiones
patron6 = r'[1-9]\s*[aA-zZ]{1,4}\/\b[aA-zZ]{1,4}\b' #Palabras divididas por / solo cuando las palabras no exceden de 4 caracteres
patron7 = r'[aA-zZ]{1,4}[A-Z]+[a-z]*[1-4]*'
#patron8 = r'\/[a-z]*[A-Z]*'


# create a list with them
regexes = [ patron3, patron4, patron5, patron6, patron7]
for i in regexes:
    generic_re = re.compile("%s|%s|%s|%s|%s" % (patron3, patron4, patron5, patron6, patron7))

In [1610]:
sf = generic_re.findall(texto)

In [1612]:
sf_clean =  [i.strip() for i in sf if i.strip() not in swords]
sf_clean

['IRC',
 'IgA',
 'II',
 '8mg/Kg',
 '1mg/Kg',
 'EEII',
 'EEII',
 'ROT',
 'RCP',
 'RMN',
 'LCR',
 'ELISA',
 'PCR',
 'HTLV',
 'LCR',
 'PCR',
 'TSP']

In [1587]:
for i in punctuation:
    sf_clean = [a.replace(i,"") for a in sf_clean]
sf_clean

['IRC',
 'IgA',
 'II',
 '8mgKg',
 '1mgKg',
 'EEII',
 'EEII',
 'ROT',
 'RCP',
 'RMN',
 'LCR',
 'ELISA',
 'PCR',
 'HTLV',
 'LCR',
 'PCR',
 'TSP',
 'MetilPredniso']

In [1549]:
texto_clean = texto.split()

In [1550]:
texto_clean = ' '.join([w for w in texto_clean if w not in swords])

In [1551]:
for i in punctuation:
    texto_clean = texto_clean.replace(i,'').lower()

In [1552]:
#texto_clean

In [1553]:
tokens = word_tokenize(texto_clean)
#tokens

In [1554]:
for sf in sf_clean:
    a = get_longform(tokens, sf)
    print(f"Esta es la short form y su forma larga\n ({sf},{a})")

Esta es la short form y su forma larga
 (EPOC,)
Esta es la short form y su forma larga
 (III,-1)
Esta es la short form y su forma larga
 (IV,-1)
Esta es la short form y su forma larga
 (NYHA,)
Esta es la short form y su forma larga
 (leucocitosmm,-1)
Esta es la short form y su forma larga
 (gdl,)
Esta es la short form y su forma larga
 (HCT,hb)
Esta es la short form y su forma larga
 (VCM,)
Esta es la short form y su forma larga
 (PLQ,)
Esta es la short form y su forma larga
 (mm,-1)
Esta es la short form y su forma larga
 (pH,)
Esta es la short form y su forma larga
 (pO257,ph)
Esta es la short form y su forma larga
 (pCO2,po257)
Esta es la short form y su forma larga
 (HCO,-1)
Esta es la short form y su forma larga
 (SatO2,)
Esta es la short form y su forma larga
 (MGIT,micobacterias)
Esta es la short form y su forma larga
 (mgd,-1)
Esta es la short form y su forma larga
 (,-1)
Esta es la short form y su forma larga
 (,-1)
Esta es la short form y su forma larga
 (leucocitosmm,-1)
Est